In [31]:
import langchain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import openai
from langchain.vectorstores import Pinecone

In [3]:
import  os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
loader = PyPDFDirectoryLoader("./pdfdocs")
docs=loader.load()
chunks = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)




In [26]:
embeddings = OpenAIEmbeddings()


In [17]:
import  os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("PINECONE_API_KEY")
print(api_key)

692811a3-35a7-43fa-a0b3-49393ba17a6c


In [22]:
# os.environ["PINECONE_API_KEY"]=api_key
# from pinecone import Pinecone

/Users/venkatadaggupati/Documents/github/GenAI/LangChain/env/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [28]:

pc = Pinecone(api_key="692811a3-35a7-43fa-a0b3-49393ba17a6c")
# index = pc.Index("langchain")

In [32]:

vectorDB= Pinecone.from_documents(chunks,embeddings,index_name="langchain")

In [33]:
def rel_docs(query,k):
    matching_results = vectorDB.similarity_search(query,k=2)
    return matching_results

In [34]:
rel_docs("what is this tool kit purpose",2)

[Document(page_content='1\n State of California Generative AI \nToolkit for Procurement, Use, and Training  \nToolkit Requirements   \nWho is this toolkit for?  \nState teams seeking to procure and evaluate  GenAI use cases. This includes people \nwith varying  roles including information security, procurement, legal, and program \ndelivery. \nToolkit Purpose  \n●This toolkit should support and empower state entities to be able to self -assess risk\nlevels, collaborate with control agencies on higher risk use cases, and document\nand share learnings throughout their GenAI experimentation.\n●This toolkit focuses  on user -centered, plain language guidelines that link out to\ndetailed policy as needed. It should not be regarded as a policy documentitself.\n●This document will be updated on an ongoing basis based on both user\nfeedback and evolving best practices in GenAI.\nToolkit Modules  \nModule  Topic  \n1    Landing Page  \n2    Where to Start  \n3    Identifying and Evaluating GenA

In [35]:
retriver = vectorDB.as_retriever()
from langchain.chains import create_retrieval_chain
#OPENAI LLM
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo")
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
        Answer the following quesions based on only the given context in triple backticks. Think twice before giving the answer.
        ```{context}```
        question:{input}
    """
)

#Chain
#Create Stuff document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)



In [36]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver,document_chain) 
response = retrieval_chain.invoke({"input":"Purpose of tool kit"})

In [38]:
response['answer']

'The purpose of the toolkit is to support and empower state entities in procuring and evaluating GenAI use cases. It aims to help state teams self-assess risk levels, collaborate with control agencies on higher risk use cases, and document and share learnings throughout their GenAI experimentation. The toolkit focuses on user-centered, plain language guidelines and will be updated based on user feedback and evolving best practices in GenAI.'

In [40]:
#Experiment with QAChain
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [46]:
llm=OpenAI()
chain=load_qa_chain(llm,chain_type="stuff")
# similar_docs=rel_docs(query,3)
query =" Tool kit usuage? Provide the response points wise"
chain.run(input_documents=rel_docs(query,3),question=query)

'\n1. The toolkit is intended to be a living resource that evolves to meet the needs of state staff navigating the complex landscape of Gen AI.\n2. In the coming months, the toolkit will be expanded with new modules, best practices, FAQs, and training courses based on user feedback and new developments in GenAI tools.\n3. Some priority areas for future content include guidance for evaluating GenAI impacts on historically vulnerable and marginalized communities, training modules for procuring and using GenAI products for state use, guidance to aid state entities in determining whether to deploy a GenAI tool, and best practices for using GenAI tools for public sector use cases.\n4. The toolkit currently contains eight modules, including a landing page, where to start, identifying and evaluating GenAI use cases, training and resources, GenAI risk assessment and management, procurement process for GenAI, toolkit roadmap and FAQ, and additional definitions and resources.\n5. The toolkit is 